In [ ]:
# Library , Dataset, and Model
#Download Pre-trained Checkpoints on Captioning

!gdown --id 1zHThOHkBbVmr4tfBk4cf_d8ib-9VzRDT

Downloading...
From: https://drive.google.com/uc?id=1zHThOHkBbVmr4tfBk4cf_d8ib-9VzRDT
To: /content/checkpoint_captioning_83000.zip
270MB [00:04, 60.8MB/s]


In [ ]:
# Unzip the Checkpoints

!unzip checkpoint_captioning_83000.zip

# Here, I used MS-COCO dataset for captioning, The dataset contains over 82,000 images, each of which has at least 5 different caption annotations. 

Archive:  checkpoint_captioning_83000.zip
   creating: checkpoints/
   creating: checkpoints/train/
  inflating: checkpoints/train/ckpt-2.data-00000-of-00001  
  inflating: checkpoints/train/checkpoint  
  inflating: checkpoints/train/ckpt-1.data-00000-of-00001  
  inflating: checkpoints/train/ckpt-1.index  
  inflating: checkpoints/train/ckpt-3.index  
  inflating: checkpoints/train/ckpt-3.data-00000-of-00001  
  inflating: checkpoints/train/ckpt-4.index  
  inflating: checkpoints/train/ckpt-2.index  
  inflating: checkpoints/train/ckpt-4.data-00000-of-00001  


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO
import numpy as np
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps
import time
import collections
import random
import os
import json

top_k = 5000
BATCH_SIZE = 64
BUFFER_SIZE = 1000
embedding_dim = 256
units = 512
vocab_size = top_k + 1
features_shape = 2048
attention_features_shape = 64

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Pick an object detection module.
* **FasterRCNN+InceptionResNet V2**: high accuracy,
* **ssd+mobilenet V2**: small and fast.

In [ ]:
module_handle = "https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1" #@param ["https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1", "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"]

detector = hub.load(module_handle).signatures['default']

In [ ]:
## Object Detection

def display_image(image):
  fig = plt.figure(figsize=(20, 15))
  plt.grid(False)
  plt.imshow(image)

def download_and_resize_image(url, new_width=256, new_height=256,
                              display=False):
  _, filename = tempfile.mkstemp(suffix=".jpg")
  response = urlopen(url)
  image_data = response.read()
  image_data = BytesIO(image_data)
  pil_image = Image.open(image_data)
  pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.ANTIALIAS)
  pil_image_rgb = pil_image.convert("RGB")
  pil_image_rgb.save(filename, format="JPEG", quality=90)
  print("Image downloaded to %s." % filename)
  if display:
    display_image(pil_image)
  return filename


def draw_bounding_box_on_image(image,ymin, xmin,
                               ymax, xmax, color,
                               font, thickness=4,
                               display_str_list=()):

  draw = ImageDraw.Draw(image)
  im_width, im_height = image.size
  (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                ymin * im_height, ymax * im_height)
  draw.line([(left, top), (left, bottom), (right, bottom), (right, top),
             (left, top)],
            width=thickness,
            fill=color)

  # If the total height of the display strings added to the top of the bounding
  # box exceeds the top of the image, stack the strings below the bounding box
  # instead of above.
  display_str_heights = [font.getsize(ds)[1] for ds in display_str_list]
  # Each display_str has a top and bottom margin of 0.05x.
  total_display_str_height = (1 + 2 * 0.05) * sum(display_str_heights)

  if top > total_display_str_height:
    text_bottom = top
  else:
    text_bottom = top + total_display_str_height
  # Reverse list and print from bottom to top.
  for display_str in display_str_list[::-1]:
    text_width, text_height = font.getsize(display_str)
    margin = np.ceil(0.05 * text_height)
    draw.rectangle([(left, text_bottom - text_height - 2 * margin),
                    (left + text_width, text_bottom)],
                   fill=color)
    draw.text((left + margin, text_bottom - text_height - margin),
              display_str,
              fill="black",
              font=font)
    text_bottom -= text_height - 2 * margin


# Draw Boxes on the image
def draw_boxes(image, boxes, class_names, scores, max_boxes=20, min_score=0.1):
  colors = list(ImageColor.colormap.values())

  try:
    font = ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Regular.ttf",
                              25)
  except IOError:
    print("Font not found, using default font.")
    font = ImageFont.load_default()

  for i in range(min(boxes.shape[0], max_boxes)):
    if scores[i] >= min_score:
      ymin, xmin, ymax, xmax = tuple(boxes[i])
      display_str = "{}: {}%".format(class_names[i].decode("ascii"),
                                     int(100 * scores[i]))
      color = colors[hash(class_names[i]) % len(colors)]
      image_pil = Image.fromarray(np.uint8(image)).convert("RGB")
      draw_bounding_box_on_image(
          image_pil,
          ymin,
          xmin,
          ymax,
          xmax,
          color,
          font,
          display_str_list=[display_str])
      np.copyto(image, np.array(image_pil))
  return image

def load_img(path):
  img = tf.io.read_file(path)
  img = tf.image.decode_jpeg(img, channels=3)
  return img
import requests

# Run Detector and Caption on the image.

def run_detector(detector, path):
  img = load_img(path)

  converted_img  = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]
  start_time = time.time()
  result = detector(converted_img)
  end_time = time.time()

  result = {key:value.numpy() for key,value in result.items()}

  print("Found %d objects." % len(result["detection_scores"]))
  print("Inference time: ", end_time-start_time)
  boxes = result["detection_boxes"][:10]
  names = result["detection_class_entities"][:10]
  score = result["detection_scores"][:10]
  human = []
  objects = []
  for i in range(len(boxes)):
    name = names[i].decode("ascii")
    print(boxes[i], score[i])
    if name == "Man" or name == "Woman":
      human.append([boxes[i], name, score[i]])
    else:
      objects.append([boxes[i], name, score[i]])
  soloObjects = []
  for j in objects:
    for k in human:
      if check_collision(j[0], k[0]):
        k[0][0] = min(j[0][0],k[0][0])
        k[0][1] = min(j[0][1],k[0][1])
        k[0][2] = max(j[0][2],k[0][2])
        k[0][3] = max(j[0][3],k[0][3])
        j.append("eliminated")
  for h in objects:
    if len(h) <= 3:
      soloObjects.append(h)
  print("Found ",len(human), " human and ", len(soloObjects)," solo objects.")
  image_with_boxes = draw_boxes(
      img.numpy(), result["detection_boxes"],
      result["detection_class_entities"], result["detection_scores"])

  display_image(image_with_boxes)
  endJson = {}
  endJson["Boxes"] = []
  endJson["Caption"] = []
  for h in human:
    endJson["Boxes"].append([str(a) for a in list(h[0])])
    cap = captioningImage(path, h[0])
    endJson["Caption"].append(cap)
  for j in soloObjects:
    endJson["Boxes"].append([str(a) for a in list(j[0])])
    cap = captioningImage(path, j[0])
    endJson["Caption"].append(cap)
  print(endJson)
  with open(path + '.json', 'w') as f:
      json.dump(endJson, f)
    
# Function to Caption Image.

def captioningImage(url, box):

  pil_image = Image.open(url)
  width, height = pil_image.size

  ymin, xmin, ymax, xmax = tuple(box)
  # Setting the points for cropped image
  left = xmin * width
  top = ymin * height
  right = xmax * width
  bottom = ymax * height
  
  # Cropped image of above dimension
  # (It will not change original image)

  pil_image = pil_image.crop((left, top, right, bottom))
  pil_image_rgb = pil_image.convert("RGB")
  pil_image_rgb.save("tmpimage", format="JPEG", quality=90)
  display_image(pil_image_rgb)
  result, attention_plot = evaluate("tmpimage")
  print('Prediction Caption:', ' '.join(result))
  return ' '.join(result)


# Function to Check Collision.

def check_collision (tobject, thuman):
  ymi,xmi,yma,xma = tuple(tobject)
  ymiH,xmiH,ymaH,xmaH = tuple(thuman)
  #If any of the sides from object are outside of human
  if xmi > xmaH: return False
  if xma < xmiH: return False
  if ymi > ymaH: return False
  if yma < ymiH: return False
  #If none of the sides from object is outside human
  return True;


In [ ]:
# Captioining Attention and Encoders for captioning.
# SHOW AND TELL MODEL
class BahdanauAttention(tf.keras.Model):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, features, hidden):
    hidden_with_time_axis = tf.expand_dims(hidden, 1)
    attention_hidden_layer = (tf.nn.tanh(self.W1(features) +
                                         self.W2(hidden_with_time_axis)))
    score = self.V(attention_hidden_layer)
    attention_weights = tf.nn.softmax(score, axis=1)
    context_vector = attention_weights * features
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

class CNN_Encoder(tf.keras.Model):
    # Since you have already extracted the features and dumped it
    # This encoder passes those features through a Fully connected layer
    def __init__(self, embedding_dim):
        super(CNN_Encoder, self).__init__()
        self.fc = tf.keras.layers.Dense(embedding_dim)

    def call(self, x):
        x = self.fc(x)
        x = tf.nn.relu(x)
        return x

class RNN_Decoder(tf.keras.Model):
  def __init__(self, embedding_dim, units, vocab_size):
    super(RNN_Decoder, self).__init__()
    self.units = units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc1 = tf.keras.layers.Dense(self.units)
    self.fc2 = tf.keras.layers.Dense(vocab_size)

    self.attention = BahdanauAttention(self.units)

  def call(self, x, features, hidden):
    # defining attention as a separate model
    context_vector, attention_weights = self.attention(features, hidden)
    x = self.embedding(x)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
    output, state = self.gru(x)
    x = self.fc1(output)
    x = tf.reshape(x, (-1, x.shape[2]))
    x = self.fc2(x)
    return x, state, attention_weights

  def reset_state(self, batch_size):
    return tf.zeros((batch_size, self.units))

embedding_dim = 256
encoder = CNN_Encoder(embedding_dim)
decoder = RNN_Decoder(embedding_dim, units, vocab_size)

optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')


def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [ ]:
# Evaluate Code on Captioning


# restoring the latest checkpoint in checkpoint_path
checkpoint_path = "/content/checkpoints/train"
ckpt = tf.train.Checkpoint(encoder=encoder,
                           decoder=decoder,
                           optimizer=optimizer)
ckpt.restore(tf.train.latest_checkpoint(checkpoint_path))
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)


start_epoch = 0
if ckpt_manager.latest_checkpoint:
  start_epoch = int(ckpt_manager.latest_checkpoint.split('-')[-1])
  ckpt.restore(ckpt_manager.latest_checkpoint)


def evaluate(image):
    attention_plot = np.zeros((max_length, attention_features_shape))

    hidden = decoder.reset_state(batch_size=1)

    temp_input = tf.expand_dims(load_image(image)[0], 0)
    img_tensor_val = image_features_extract_model(temp_input)
    img_tensor_val = tf.reshape(img_tensor_val, (img_tensor_val.shape[0],
                                                 -1,
                                                 img_tensor_val.shape[3]))

    features = encoder(img_tensor_val)

    dec_input = tf.expand_dims([tokenizer.word_index['<start>']], 0)
    result = []

    for i in range(max_length):
        predictions, hidden, attention_weights = decoder(dec_input,
                                                         features,
                                                         hidden)

        attention_plot[i] = tf.reshape(attention_weights, (-1, )).numpy()

        predicted_id = tf.random.categorical(predictions, 1)[0][0].numpy()
        result.append(tokenizer.index_word[predicted_id])

        if tokenizer.index_word[predicted_id] == '<end>':
            return result, attention_plot

        dec_input = tf.expand_dims([predicted_id], 0)

    attention_plot = attention_plot[:len(result), :]
    return result, attention_plot

annotation_folder = '/annotations/'
if not os.path.exists(os.path.abspath('.') + annotation_folder):
  annotation_zip = tf.keras.utils.get_file('captions.zip',
                                           cache_subdir=os.path.abspath('.'),
                                           origin='http://images.cocodataset.org/annotations/annotations_trainval2014.zip',
                                           extract=True)
  os.remove(annotation_zip)

annotation_file = '/content/annotations/captions_train2014.json'
with open(annotation_file, 'r') as f:
    annotations = json.load(f)
train_captions = []

for val in annotations['annotations']:
  caption = f"<start> {val['caption']} <end>"
  train_captions.append(caption)

image_model = tf.keras.applications.InceptionV3(include_top=False,
                                                weights='imagenet')
new_input = image_model.input
hidden_layer = image_model.layers[-1].output

image_features_extract_model = tf.keras.Model(new_input, hidden_layer)

top_k = 5000
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=top_k,
                                                  oov_token="<unk>",
                                                  filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~')
tokenizer.fit_on_texts(train_captions)

def load_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (299, 299))
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    return img, image_path

tokenizer.word_index['<pad>'] = 0
tokenizer.index_word[0] = '<pad>'

def calc_max_length(tensor):
    return max(len(t) for t in tensor)
train_seqs = tokenizer.texts_to_sequences(train_captions)
max_length = calc_max_length(train_seqs)

87924736/87910968 [==============================] - 1s 0us/step


In [ ]:
# Download one Image
!wget --output-document=test.jpg https://variety.com/wp-content/uploads/2019/09/modern-love.jpg?w=1000

--2021-09-29 02:28:41--  https://variety.com/wp-content/uploads/2019/09/modern-love.jpg?w=1000
Resolving variety.com (variety.com)... 192.0.66.176
Connecting to variety.com (variety.com)|192.0.66.176|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 362574 (354K) [image/jpeg]
Saving to: ‘test.jpg’

test.jpg            100%[===================>] 354.08K  1.22MB/s    in 0.3s    

2021-09-29 02:28:42 (1.22 MB/s) - ‘test.jpg’ saved [362574/362574]



In [ ]:
# Run the Detector on the Image, a corresponded Json will be saved
run_detector(detector, "test.jpg")  